### 🛠️ Initialize Notebook Variables

**Only modify entries under _USER CONFIGURATION_.**

In [ ]:
import utils
from apimtypes import *

# ------------------------------
#    USER CONFIGURATION
# ------------------------------

rg_location = 'eastus2'
index       = 1
apim_sku    = APIM_SKU.BASICV2              # Options: 'BASICV2', 'STANDARDV2', 'PREMIUMV2'
deployment  = INFRASTRUCTURE.SIMPLE_APIM    # Options: 'AFD_APIM_PE', 'APIM_ACA', 'SIMPLE_APIM'
api_prefix  = ''            # Not defining a prefix for general as these APIs will live off the root
tags        = ['general']



# ------------------------------
#    SYSTEM CONFIGURATION
# ------------------------------

sample_folder    = 'general'
rg_name          = utils.get_infra_rg_name(deployment, index)
supported_infras = [INFRASTRUCTURE.AFD_APIM_PE, INFRASTRUCTURE.APIM_ACA, INFRASTRUCTURE.SIMPLE_APIM]
nb_helper        = utils.NotebookHelper(sample_folder, rg_name, rg_location, deployment, supported_infras, index = index, apim_sku = apim_sku)

# Define the APIs and their operations and policies

# API 1
api1_path = f'{api_prefix}api1'
api1_get  = GET_APIOperation('This is a GET for API 1')
api1_post = POST_APIOperation('This is a POST for API 1')
api1      = API(api1_path, 'API 1', api1_path, 'This is API 1', operations = [api1_get, api1_post], tags = tags)

# API 2
api2_path = f'{api_prefix}api2'
api2_post = POST_APIOperation('This is a POST for API 2')
api2      = API(api2_path, 'API 2', api2_path, 'This is API 2', operations = [api2_post], tags = tags)

# API 3: Request Headers
rh_path                 = f'{api_prefix}request-headers'
pol_request_headers_get = utils.read_policy_xml(REQUEST_HEADERS_XML_POLICY_PATH)
request_headers_get     = GET_APIOperation('Gets the request headers for the current request and returns them. Great for troubleshooting.', pol_request_headers_get)
request_headers         = API(rh_path, 'Request Headers', rh_path, 'API for request headers', operations = [request_headers_get], tags = tags)

# APIs Array
apis: List[API] = [api1, api2, request_headers]

utils.print_ok('Notebook initialized')

### 🚀 Deploy Infrastructure and APIs

Creates the bicep deployment into the previously-specified resource group. A bicep parameters, `params.json`, file will be created prior to execution.

In [ ]:
# Build the bicep parameters
bicep_parameters = {
    'apis': {'value': [api.to_dict() for api in apis]}
}

# Deploy the sample
output = nb_helper.deploy_sample(bicep_parameters)

if output.success:
    # Extract deployment outputs for testing
    apim_gateway_url = output.get('apimResourceGatewayURL', 'APIM Gateway URL')
    apim_apis        = output.getJson('apiOutputs', 'APIs')

    utils.print_ok('Deployment completed successfully')
else:
    utils.print_error("Deployment failed!")
    raise SystemExit(1)

### ✅ Verify Deployment

Assert that the deployment was successful by making simple calls to APIM. 

❗️ If the infrastructure shields APIM and requires a different ingress (e.g. Azure Front Door), the request to the APIM gateway URl will fail by design. Obtain the Front Door endpoint hostname and try that instead.

In [ ]:
from apimrequests import ApimRequests
from apimtesting import ApimTesting

# Initialize testing framework
tests = ApimTesting("General Sample Tests", sample_folder, nb_helper.deployment)
api_subscription_key = apim_apis[2]['subscriptionPrimaryKey']

# Preflight: Check if the infrastructure architecture deployment uses Azure Front Door. If so, assume that APIM is not directly accessible and use the Front Door URL instead.
endpoint_url = utils.test_url_preflight_check(deployment, rg_name, apim_gateway_url)

# 1) Request Headers
reqs = ApimRequests(endpoint_url, api_subscription_key)
output = reqs.singleGet('/request-headers', msg = 'Calling Request Headers API via Azure Front Door. Expect 200.')
tests.verify('Host:' in output, True)

tests.print_summary()

utils.print_ok('All done!')